# Projet Mapping

In [6]:
#!pip install biopython
from Bio import SeqIO 
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import numpy as np
import sys

## Lecture des données

On dispose de la séquence du génome de Plasmodium falciparum sur 14 chromosomes.

In [7]:
Genome = list(SeqIO.parse('data','fasta'))

In [8]:
print(str(Genome[0].seq)[:100])
print('Chromosome 1 length :',len(Genome[0].seq))
print('Chromosome 2 length :',len(Genome[1].seq))
print('Chromosome 14 length :',len(Genome[13].seq))

TGAACCCTaaaacctaaaccctaaaccctaaaccctgaaccctaaaccctgaaccctaaaccctaaaccctgaaccctaaaccctaaaccctgaacccta
Chromosome 1 length : 640851
Chromosome 2 length : 947102
Chromosome 14 length : 3291936


In [9]:
reads = []  
for record in SeqIO.parse("single_Pfal_dat.fq", "fastq"):
    reads.append(record)

In [10]:
print(str(reads[3].seq))
print('Reads length :',len(reads[2].seq))
print('Number of reads :',len(reads))

AATTAATAATTATTATTATTACATATATATTTGTTATGTTTTGTTATATAATATTTAATATGAAGTATTATATAAATATATATATATATGTGTGCCATGT
Reads length : 100
Number of reads : 1500000


Pour élaborer votre méthode de mapping, il est nécessaire de pouvoir lire et analyser 
des fichiers de données de séquençage. La première étape du projet consiste donc à prendre 
en main la bibliothèque Biopython pour manipuler les fichiers au format FASTQ.

Dans un second temps, il vous faudra définir une structure de données permettant de 
chercher des mots de longueur fixe (k > 0) à partir d’un texte t de longueur n et capable de 
répondre efficacement aux questions suivantes : 
-  Étant donné un mot w de longueur k, est-il présent (lui ou son complémentaire inversé) dans le texte indexé t ? 
    - le cas échéant, combien de fois apparaît-il (quel est son support), à quelles positions et sur quel brin ? 
 
-  Quel est le mot de longueur k présent à la position i (0<i<=n) dans le texte t ?
 
-  Bonus : proposez une méthode de recherche de sous-séquences approchée autorisant l’alignement de sous-séquences avec un nombre maximal d’erreurs d. 
 
   
Dans le choix de la structure, vous veillerez bien à prendre en compte le fait que : 
1.  le texte t est formé sur un petit alphabet (A, C, G et T) 
2.  le texte t peut-être très grand (n > 109) 
3.  une structure de données peut être constituée de plusieurs structures distinctes.

## Implémentation des fonctions

In [11]:
def complementInverse(w):
    wCI = []
    for i in range(len(w)):
        li = w[-i-1]
        if li not in "ATCG":
            raise Exception("Letter not in alphabet")
        lCI = 'A'*int(li=='T') + 'T'*int(li=='A') + 'C'*int(li=='G') + 'G'*int(li=='C')
        wCI.append(lCI)
    return ''.join(wCI)

In [12]:
# Test fonction complementInverse

testWord = "ATCG"
ansWord = "CGAT"
ans = complementInverse(testWord)
if complementInverse(testWord) != ansWord:
    raise Exception("Wrong result")

In [13]:
def circularShift(T,k):
    if k<0:
        raise Exception("k must be positive")
    return T[k:]+T[0:k]

def BWT(S):
    if S[len(S)-1]!='$':
        S += '$'
        
    circShifts = []
    for k in range(len(S)):
        circShifts.append(circularShift(S,k))
    circShifts.sort()
    
    bwt = []
    for shift in circShifts:
        bwt.append(shift[len(S)-1])
        
    return bwt

In [14]:
# Test fonction BWT

testText = "ATGCCTGATCG"
testTextBWT = BWT(testText)
testTextBWT

['G', 'G', '$', 'G', 'T', 'C', 'C', 'T', 'T', 'A', 'C', 'A']

In [15]:
def suffix_list(T): # O(n log n)
    """
    Compute the suffix list
    
    Args:
        T (str): string
    
    Return:
        list of strings: suffix list
    """
    suffix_list = [T[i:] for i in range(len(T))] # O(n)
    sorted(suffix_list,reverse=True) # O(n log n)
    return suffix_list

def suffix_table(T): # O(n log n)
    """
    Compute the suffix table
    
    Args:
        T (str): string
    
    Return:
        list of tuples (suffix,location): suffix table
    """
    suffix_list = [T[i:] for i in range(len(T))] 
    suffix_table = sorted((e,i) for i,e in enumerate(suffix_list))
    return suffix_table

def BWT_suffix_table(T,end_of_string="$"): # O(n log n)
    """
    Compute the BWT from the suffix table
    
    Args:
        T (str): string
        end_of_string (char): end of string character to append
    
    Return:
        bwt (str): BWT
    """
    if T[-1]!=end_of_string :
        T += end_of_string
    ST = suffix_table(T) # O(n log n)
    bwt = ""
    index = []
    for s,i in ST: # O(n)
        bwt += T[i-1]
        index.append(i-1)
    return bwt, index

Let `n` the size of `T`.

Complexity of `suffix_list(T)` :

In [16]:
suffix_list(testText)

['ATGCCTGATCG',
 'TGCCTGATCG',
 'GCCTGATCG',
 'CCTGATCG',
 'CTGATCG',
 'TGATCG',
 'GATCG',
 'ATCG',
 'TCG',
 'CG',
 'G']

In [17]:
suffix_table(testText)

[('ATCG', 7),
 ('ATGCCTGATCG', 0),
 ('CCTGATCG', 3),
 ('CG', 9),
 ('CTGATCG', 4),
 ('G', 10),
 ('GATCG', 6),
 ('GCCTGATCG', 2),
 ('TCG', 8),
 ('TGATCG', 5),
 ('TGCCTGATCG', 1)]

In [18]:
# Test fonction BWT_suffix_table

BWT_suffix_table(testText)

('GG$GTCCTTACA', [10, 6, -1, 2, 8, 3, 9, 5, 1, 7, 4, 0])

In [19]:
from collections import Counter 

def occurrence_indexer(S):
    """
    Number of past occurrences of each char in S
    
    Args:
        S (str): string
        
    Return:
        table of int : number of past occurrences of the char
    """
    K = []
    last_index = {}
    for s in S:
        if s not in last_index: 
            last_index[s] = 0
        K.append(last_index[s])
        last_index[s] += 1
    return(K)

def last2first(counts,k,X):
    """
    k + nbr of occurrences of letters < X 
    
    Args:
        counts (Counter): nbr of each char in the text
        k : int
        X : char
        
    Return:
        int : k + nbr of occurrences of letters < X 
    """
    a = [counts[char] for char in counts if char < X]
    return k + sum(a)
    
def efficient_inverse_BWT(bwt,end_of_string="$"):
    """
    Inverse the BWT
    
    Args:
        bwt (str): bwt of a string T
        last_character (char): which is the end of string character?
    
    Return:
        T (str): BWT^{-1} of bwt
    """    
    K = occurrence_indexer(bwt)
    counts = Counter(bwt)
    X = bwt[0]
    k = K[0]
    S = end_of_string
    while X != end_of_string:
        S = X+S
        j = last2first(counts,k,X)
        X = bwt[j]
        k = K[j]
    return(S)

In [20]:
# Test fonction occurrence_indexer

occurrence_indexer(testText)

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2]

In [ ]:
# Test fonction last2first

print(testText)
last2first(Counter(testText),0,"T")

ATGCCTGATCG


0

In [22]:
#efficient_inverse_BWT(BWT_suffix_table(testText)) == testText + "$"

In [23]:
  def get_first_occurrence(L,X):
    for i,l in enumerate(L):
        if l == X:
            return(i)
        
def pattern_matching_BWT(T,pattern):
    """
    Search a patter in a String using the BWT
    
    Args:
        S (str): string
        pattern (str): pattern
    
    Return:
        bool: true if the pattern is in the string    
    """
    L,_ = BWT_suffix_table(T)
    K = occurrence_indexer(L)
    counts = Counter(L)
    e = 0
    f = len(L)
    i = len(pattern) - 1
    while e < f and i >= 0:
        X = pattern[i]
        first_occurence_in_L_ef = get_first_occurrence(L[e:f],X)
        if first_occurence_in_L_ef is None:
            return False
        else:
            r = first_occurence_in_L_ef+e
        #print(first_occurence_in_L_ef,L[e:f],r,X,L[r])
        
        last_occurence_in_L_ef = get_first_occurrence(L[e:f][::-1],X)
        if last_occurence_in_L_ef is None:
            return False
        else:
            s = f-last_occurence_in_L_ef-1
        #print(last_occurence_in_L_ef,L[e:f],s,X,L[s])
        
        e = last2first(counts,K[r],X)
        f = last2first(counts,K[s],X)+1
        i -= 1
        #print(r,s,e,f)
    return(i<0)

In [24]:
# Test fonction pattern_matching_BWT

print(testWord, 'in', testText)
pattern_matching_BWT(testText,testWord)

ATCG in ATGCCTGATCG


True

In [25]:
def rankBWT(bwt):
    """
    Number of past occurrences of each char in S
    
    Args:
        S (str): string
        
    Return:
        table of int : number of past occurrences of the char
    """
    K = []
    tot = {}
    for s in bwt:
        if s not in tot: 
            tot[s] = 0
        K.append(tot[s])
        tot[s] += 1
    return(K,tot)
    

In [26]:
# Test fonction rankBWT

testBWT,_ = BWT_suffix_table(testText)
rank,tots = rankBWT(testBWT)

In [27]:
def invBWT(bwt,A):
    A = sorted(A)
    #count = {a:0 for a in A}
    count = {}
    K = []
    for i in range(len(bwt)):
        if bwt[i] in count:
            count[bwt[i]] += 1
        else:
            count[bwt[i]] = 1
        K.append(count[bwt[i]])
    X = bwt[0]
    k = K[1]
    S = '$'
    while X != '$':
        S = X+S
        j = k
        i = 0
        while A[i] != X:
            j += count[A[i]]
            i += 1
        X = bwt[j]
        k = K[j]
    return S

In [28]:
def firstColMap(tots):
    """ Return a map from characters to the range of cells in the first
    column containing the character. 
    char : (first index, last index+1)"""
    first = {}
    totc = 0
    for c, count in sorted(tots.items()):
        first[c] = (totc, totc + count)
        totc += count
    return first

In [29]:
def searching(bwt, SAT, W):
    
    # Not case sensitive, make everything uppercase
    bwt = bwt.upper()
    W = W.upper()
    
    # Perdorm a BWT on text T and get the indexes corresponding to the Suffix Array of T
    #bwt, SAT = BWT_suffix_table(T)
    # LF mapping
    # F = sorted(bwt) # no need to store it
    rank, totCount = rankBWT(bwt)
    Fmap = firstColMap(totCount)
    del totCount
    
    #print('\nbwt :', sys.getsizeof(bwt),
    #'\nSAT :', sys.getsizeof(SAT),
    #'\nrank :', sys.getsizeof(rank),
    #'\nFmap :', sys.getsizeof(Fmap),)

    letter = W[-1] # last letter of the word W
    if letter not in Fmap:
        return -1, "Word not in text"
    fi, li = Fmap[letter] # find occurences of letter in F
    
    for i in range(2,len(W)+1):
        letter = W[-i] # search W by going backward through it
        if letter not in Fmap:
            return -1, "Word not in text"

        j = fi
        foundRankf = False
        while j < li:
            if bwt[j] == letter:
                if not foundRankf:
                    rankf = rank[j] # get the rank of the first occurence of letter (followed by W suffix)
                    foundRankf = True
                rankl = rank[j] # get the rank of the last occurence of letter (followed by W suffix)
            j += 1
        
        if not foundRankf:
            return -1, "Word not in text"
        
        fi, li = Fmap[letter] # find occurences of letter in F
        fi += rankf # find first occurence of letter in F (followed by W suffix)
        li = fi + rankl-rankf +1 # find last occurence of letter in F (followed by W suffix)
        
    return [x+1 for x in SAT[fi:li]]
    

In [30]:
# Test fonction searching

BWTpwet, SATpwet = BWT_suffix_table('pwetPWetpwotpwitpwet')
searching(BWTpwet, SATpwet, 'PWET')

(-1, 'Word not in text')

In [31]:
# sWord = 'c'
# s10 = searching(str(s)[0:10], sWord)
# s100 = searching(str(s)[0:100], sWord)
# s1000 = searching(str(s)[0:1000], sWord)

## Browse reads

Découpons le premier chromosome en bouts de longueur `n` avec un overlap de `k`.

In [32]:
def cutSequence(seqToCut, n, k):
    cutSeq = []
    iF = 0
    iL = iF + n

    Continuer = True

    while Continuer:
        if iL >= len(seqToCut):
            Continuer = False
            iL = len(seqToCut)
        cutSeq.append(seqToCut[iF:iL])
        iF += n - k
        iL = iF + n -1
        if Continuer and len(seqToCut) - iF <= k :
            Continuer = False
            
    return cutSeq

In [33]:
def searchWord(cutSeq, n, k, words):

    bwtSeq = []
    satSeq = []
    for s in cutSeq:
        bwt, sat = BWT_suffix_table(s)
        bwtSeq.append(bwt)
        satSeq.append(sat)
        
    foundRead = []
    for i in range(len(cutSeq)):
        res = searching(bwtSeq[i], satSeq[i], words)
        if res[0] != -1 :
            resi = [x+i*(n-k) for x in res]
            foundRead += resi
            
    return np.unique(foundRead)

In [34]:
# Test fonction cutSequence

seqToCut = 'abaghstbababbagtdjaba'
n = 11
words = 'aba'
k = len(words)

repTest = [0,8,18]
cutSeq = cutSequence(seqToCut, n, k)
cutSeq

['abaghstbaba', 'ababbagtdj', 'djaba']

In [35]:
# Test fonction foundWord

foundWord = searchWord(cutSeq, n, k, words)
foundWord

array([ 0,  8, 18])

In [36]:
for i in foundWord:
    print(seqToCut[i-1:i-1+k])


bab
jab


Les fonctions fonctionnent sur la séquence test. Essayons sur le chromosome 1 :

In [37]:
chr1 = Genome[0].seq.upper()
n = int(1e4)
read1 = reads[0].seq.upper()
k1 = len(read1)
k1

100

In [38]:
cutChr1 = cutSequence(chr1, n, k1)
cutChr1

[Seq('TGAACCCTAAAACCTAAACCCTAAACCCTAAACCCTGAACCCTAAACCCTGAAC...TAC', SingleLetterAlphabet()),
 Seq('TGACTACTAACATGATCAGTAACATGACTACTAACATCATCATAACTAACATGA...CTG', SingleLetterAlphabet()),
 Seq('GATCTTACTTTCACTAACTTAGGTCTTACTTTTGCTAACATAGGTCTTACTTTC...AGA', SingleLetterAlphabet()),
 Seq('CTATAAACCATGTGCCCTTGAATATGAATATTATAAGCATACTAATGGCGGTGG...TTA', SingleLetterAlphabet()),
 Seq('GGAATGGAGTCACTTTGTATATCAGTTTGTAATGTGGCAAATTTGTCCATTAAT...TAA', SingleLetterAlphabet()),
 Seq('ACACTACATAAAACTATACAATAGTATTTTATTAATCTTAATAATTTCTTTTTT...GAT', SingleLetterAlphabet()),
 Seq('TATTGTAAAATATAAAACAATATTTTAATGGTAATTTATCGTATGACATAAAAT...GCT', SingleLetterAlphabet()),
 Seq('TATTTATGTATCACTTTTATTTTCTGTTTTATCTTTACCCATCAATTTCCCTCT...AAA', SingleLetterAlphabet()),
 Seq('GTACCAGATAAACAAGGTTACTTACGACATAAATTTGAATGTTACCGAAAATAT...AGT', SingleLetterAlphabet()),
 Seq('ATTACATATATTTATTGTATTTATATAATAAAAAATATAATATATTATTTGTAC...TTA', SingleLetterAlphabet()),
 Seq('TATCATTTTAAATGTAAGTTTTTTTTTTTTTTTTTTTTTTTGAAATAAAATACA

In [39]:
foundRead1 = searchWord(cutChr1, n, k1, read1)

In [40]:
foundRead1

array([], dtype=float64)

On va faire une seule fonction pour rassembler les deux étapes (cut et search)

In [41]:
def cut_search(seqToCut, n, word):
    k = len(word)
    cutSeq = cutSequence(seqToCut, n, k)
    foundWord = searchWord(cutSeq, n, k, word)
    return foundWord

On va chercher le premier read dans chaque chromosome

In [43]:
n = int(1e4)
read1 = reads[0].seq.upper()
foundRead = {i:[] for i in range(len(Genome))}

for i in range(len(Genome)):
    chromosome = Genome[i].seq.upper()
    found = cut_search(chromosome, n, read1)
    print(found)
    foundRead[i] = found

KeyboardInterrupt: 

In [ ]:
foundRead

{0: array([], dtype=float64),
 1: array([], dtype=float64),
 2: array([], dtype=float64),
 3: array([], dtype=float64),
 4: array([], dtype=float64),
 5: array([], dtype=float64),
 6: array([], dtype=float64),
 7: array([], dtype=float64),
 8: array([], dtype=float64),
 9: array([], dtype=float64),
 10: array([], dtype=float64),
 11: array([], dtype=float64),
 12: array([], dtype=float64),
 13: array([], dtype=float64),
 14: array([], dtype=float64)}

Y'a R

In [46]:
print(reads[0].description, 'position is 131735')
print(str(Genome[0].seq.upper())[131734:131834],complementInverse(reads[0].seq))
pattern_matching_BWT(str(Genome[0].seq.upper())[128700:140000],complementInverse(reads[0][1:14].seq))

NC_004325.2-100000 position is 131735
TTAATATATTCCTCATATATTTATTTATATGGATCTTTTCACCCGTTACTATATAATACTATATTTTTTTTATTAAAAAACAAAACGCTTAAAAAGGAAA TTAATATATTCCTCATATATTTATTTATATGGATCTTTTCACCCGTTACTATATAATACTATATTTTTTTTATTAAAAAATAAAACGCTTAAAAAGGAAA


True

Premier Read sur le premier chromosome --> pb :
Pris en compte un % de match de bp? Testé sur chaque brin dans les 2 sens?

In [ ]:
n = int(1e4)
foundAll = {j:{} for j in range(len(reads))}

for j in range(len(reads)):
    print("read ",j)
    read = reads[j].seq.upper()
    foundRead = {i:[] for i in range(len(Genome))}

    for i in range(len(Genome)):
        print("chr",i+1)
        chromosome = Genome[i].seq.upper()
        found = cut_search(chromosome, n, read)
        print(found)
        foundRead[i] = found
        
    foundAll[j] = foundRead

read  0
chr 1
[]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  1
chr 1
[]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  2
chr 1
[471737]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  3
chr 1
[]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  4
chr 1
[]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  5
chr 1
[]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  6
chr 1
[592074]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[1390936]
chr 8
[]
chr 9
[

In [ ]:
n = int(1e4)
foundWord = {i:{} for i in range(len(Genome))}
foundInv = {i:{} for i in range(len(Genome))}
k = len(reads[0])

for i in range(len(Genome)):
    print("chr",i+1)
    chromosome = Genome[i].seq.upper()
    cutChr = cutSequence(chromosome, n, k)
    
    for j in range(len(reads)):
        print("read ",j)
        read = reads[j].seq.upper()
        inv = complementInverse(read)
        
        foundWord_ = searchWord(cutChr, n, k, read)
        foundInv_ = searchWord(cutChr, n, k, inv)
        
        if len(foundWord_) != 0:
            foundWord[i] = (j,foundWord_)
            print((j,foundWord_))
        if len(foundInv_) != 0:
            foundInv[i] = (j,foundInv_)
            print((j,foundInv_))
        

chr 1
read  0
read  1
read  2
(2, array([471737]))
read  3
read  4
read  5
read  6
(6, array([592074]))
(6, array([82780]))
read  7
(7, array([270169]))
read  8
read  9
read  10
read  11
(11, array([21639]))
read  12
(12, array([56757]))
read  13
(13, array([580626]))
read  14
read  15
(15, array([607662]))
read  16
(16, array([121139]))
read  17
(17, array([418998]))
read  18
read  19
(19, array([133661]))
read  20
read  21
read  22
(22, array([38626, 81407]))
read  23
read  24
read  25
read  26
read  27
(27, array([328241]))
read  28
read  29
(29, array([504603]))
read  30
(30, array([344258]))
read  31
(31, array([294113]))
read  32
(32, array([75561]))
read  33
read  34
read  35
(35, array([181919]))
read  36
read  37
read  38
read  39
read  40
(40, array([527402]))
read  41
read  42
read  43
read  44
(44, array([393825]))
read  45
(45, array([546247]))
read  46
read  47
(47, array([90618]))
read  48


Génome : 23Mpb

1.5M reads de 100pb soit 150Mpb à mapper

Ça fonctionne mais c'est turbo long !

In [ ]:
n = int(1e4)
read1 = reads[31828].seq.upper()
foundRead = {i:[] for i in range(len(Genome))}

for i in range(len(Genome)):
    chromosome = Genome[i].seq.upper()
    found = cut_search(chromosome, n, read1)
    print(found)
    foundRead[i] = found
    break

[253370]


On va couper tout le génome

In [ ]:
wholeGenomeCutted = []
n = int(1e4)
k = len(reads[0])

for i in range(len(Genome)):
    chromosome = Genome[i].seq.upper()
    cutChr = cutSequence(chromosome, n, k)
    wholeGenomeCutted.append(cutChr)

In [ ]:
wholeGenomeCutted

[[Seq('TGAACCCTAAAACCTAAACCCTAAACCCTAAACCCTGAACCCTAAACCCTGAAC...TAC'),
  Seq('TGACTACTAACATGATCAGTAACATGACTACTAACATCATCATAACTAACATGA...CTG'),
  Seq('GATCTTACTTTCACTAACTTAGGTCTTACTTTTGCTAACATAGGTCTTACTTTC...AGA'),
  Seq('CTATAAACCATGTGCCCTTGAATATGAATATTATAAGCATACTAATGGCGGTGG...TTA'),
  Seq('GGAATGGAGTCACTTTGTATATCAGTTTGTAATGTGGCAAATTTGTCCATTAAT...TAA'),
  Seq('ACACTACATAAAACTATACAATAGTATTTTATTAATCTTAATAATTTCTTTTTT...GAT'),
  Seq('TATTGTAAAATATAAAACAATATTTTAATGGTAATTTATCGTATGACATAAAAT...GCT'),
  Seq('TATTTATGTATCACTTTTATTTTCTGTTTTATCTTTACCCATCAATTTCCCTCT...AAA'),
  Seq('GTACCAGATAAACAAGGTTACTTACGACATAAATTTGAATGTTACCGAAAATAT...AGT'),
  Seq('ATTACATATATTTATTGTATTTATATAATAAAAAATATAATATATTATTTGTAC...TTA'),
  Seq('TATCATTTTAAATGTAAGTTTTTTTTTTTTTTTTTTTTTTTGAAATAAAATACA...TTT'),
  Seq('TAAATATTTATTTATTTTAAGTGGAAATATGCGGAAATTATTGAAAAAAAAAAA...TTG'),
  Seq('AAAATATAAATATAAATATAAAGAAAAATATAAAGAAAAATATAAAAAAACAAT...GTA'),
  Seq('TTATTTTCCTCTTAGTTTAATTTAAATAAAAATGGGTAAACAAGCATAACACAA...ATC'),
  Seq(

## BAM

In [ ]:
#!pip install pysam
import pysam

In [ ]:
samfile = pysam.AlignmentFile('./single_Pfal_dat.bam', 'rb')

In [ ]:
allMap = samfile.fetch(until_eof=True)

In [ ]:
n = int(1e4)
nMap = 1
i = 0
for mapp in allMap:
    print(mapp.qname)

    i += 1
    
        
    print(mapp.pos)
    # on va chercher la lecture correspondante 
    for j in range(len(reads)):
        if reads[j].id == mapp.qname:
            jSearch = j
            readSearch = reads[j].seq
            break
            
    print('Read n°',j) 
    print(reads[jSearch].id)
    foundRead = []
    
    # On vas rechercher le read dans le génome complet
    for k in range(len(Genome)):
        print('Chr',k)
        cutChr = wholeGenomeCutted[k]
        found = searchWord(cutChr, n, len(readSearch), readSearch)
        if len(found) !=0 :
            print(found)
            foundRead.append((k,found))
            break
    if i>=nMap:
        break
        

NC_004325.2-99988
56757
Read n° 12
NC_004325.2-99988
Chr 0
[56757]


In [ ]:
mapp.qname

'NC_004325.2-99988'

In [ ]:
reads[jSearch]

SeqRecord(seq=Seq('ATGTATCATAATACTTTATAAAAGCATAAATACAATTTTTCATTTCATCAGTTG...AAT'), id='NC_004325.2-99988', name='NC_004325.2-99988', description='NC_004325.2-99988', dbxrefs=[])

In [ ]:
foundRead

[(0, array([56757]))]

In [ ]:
print(mapp)

NC_004325.2-99988	0	#0	56758	99	100=	*	0	0	ATGTATCATAATACTTTATAAAAGCATAAATACAATTTTTCATTTCATCAGTTGATTTTCCATCTTTAACTAATTTATAAAAATTAAGAGTATGTTCAAT	array('B', [34, 25, 34, 37, 37, 37, 33, 37, 37, 39, 39, 38, 38, 41, 41, 36, 25, 39, 38, 41, 38, 38, 40, 40, 40, 36, 10, 40, 34, 11, 34, 41, 40, 25, 40, 41, 35, 39, 39, 31, 41, 41, 40, 38, 30, 40, 40, 35, 41, 41, 40, 41, 17, 41, 40, 41, 30, 41, 39, 35, 35, 33, 40, 41, 37, 41, 28, 40, 41, 33, 6, 23, 35, 39, 39, 35, 35, 35, 30, 37, 35, 35, 31, 35, 33, 35, 33, 35, 37, 37, 35, 34, 34, 34, 35, 34, 30, 35, 34, 35])	[]


In [ ]:
allMap_it

In [ ]:
allMap_it